In [27]:
#Import necessary libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from nltk.corpus import stopwords
import string
import nltk
import textmining
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
# Set woking directory
os.chdir('../robinredhu/Desktop/kaggle/ToxicCommentProject')

In [99]:
# Load the dataset
train = pd.read_csv('../ToxicCommentProject/train1.csv')
test = pd.read_csv('../ToxicCommentProject/test1.csv')
sample = pd.read_csv('../ToxicCommentProject/sample_submission.csv')

In [100]:
# Check the dimensions of dataset
print(train.shape)
print(test.shape)
print(sample.shape)

(127657, 8)
(6040, 8)
(153164, 7)


In [101]:
train.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""""\nMore\nI can't make any real suggestions on...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""""\n\nCongratulations from me as well, use the...",0,0,0,0,0,0
6,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
7,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0
8,0005300084f90edc,"""""\nFair use rationale for Image:Wonju.jpg\n\n...",0,0,0,0,0,0
9,00054a5e18b50dd4,bbq \n\nbe a man and lets discuss it-maybe ove...,0,0,0,0,0,0


In [102]:
# Viewing first seven observations of training dataset
sample.head(7)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0.5,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,0.5,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,0.5,0.5,0.5,0.5,0.5,0.5
5,0001ea8717f6de06,0.5,0.5,0.5,0.5,0.5,0.5
6,00024115d4cbde0f,0.5,0.5,0.5,0.5,0.5,0.5


In [103]:
#Check test dataset also
test.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
1,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0
2,003217c3eb469ba9,Hi! I am back again!\nLast warning!\nStop undo...,1,0,0,1,0,0
3,005f59485fcddeb0,"""""\n\nSORRY PUCK BUT NO ONE EVER SAID DICK WAS...",1,0,0,0,0,0
4,00637960a7ec3436,"""""\n\nUNBLOCK ME OR I'LL GET MY LAWYERS ON TO ...",1,0,0,0,0,0
5,0086998b34865f93,"Fuck you, block me, you faggot pussy!",1,0,1,0,1,0
6,011d0c0a6c8e5765,I NEVER FUCKING MADE THIS MOTHER FUCKING ARTIC...,1,0,1,0,1,0
7,0125f95ac41f6cea,Add Mobile porn as additional WP:ALSO link,1,0,0,0,0,0
8,013ad5246f4b953b,F**K ALL RIDES AT WORLDS OF FUN \n\nI hate al...,1,0,1,0,1,0
9,0152f55962c4f707,I don't care what you say here. I don't believ...,1,0,0,0,0,0


In [104]:
# Calculating missing values in trainning set
miss_val = pd.DataFrame(train.isnull().sum())
miss_val

,0
id,0
comment_text,0
toxic,0
severe_toxic,0
obscene,0
threat,0
insult,0
identity_hate,0


In [105]:
# Calculating missing values in testing set
miss_val2 = pd.DataFrame(train.isnull().sum())
miss_val2

,0
id,0
comment_text,0
toxic,0
severe_toxic,0
obscene,0
threat,0
insult,0
identity_hate,0


In [106]:
# Creating a function that will replace abbriviated text with normal text
def clean_text(text):
    text = re.sub(r"What's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"I'm", "I am ", text)
    text = re.sub(r"\'re", ' are ', text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\W'", " ", text)
    text = re.sub(r'\s+', " ", text)
    text = text.strip(' ')
    return text

In [107]:
# Calling clean_text function for abbreviation removal
tr_clean = []
test_clean = []
for i in range(0, len(train)):
    clean = clean_text(train.comment_text[i])
    tr_clean.append(clean)
for i in range(0, len(test)):
    clean = clean_text(test.comment_text[i])
    test_clean.append(clean)
train.comment_text = pd.Series(tr_clean).astype(str)
test.comment_text = pd.Series(test_clean).astype(str)

In [108]:
# Extract all stopwords available in the library
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)

In [109]:
# Defining a function to remove stopwords 
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = "".join([i for i in stop_free if i not in exclude])
    num_free = "".join([i for i in punc_free if not i.isdigit()])
    return num_free
post_corpus = [clean(train.iloc[i,1]) for i in range(0,train.shape[0])]
post_corpus2 = [clean(test.iloc[i,1]) for i in range(0,test.shape[0])]

In [110]:
# Replacing uncleaned text by cleaned text.
train.comment_text = post_corpus
test.comment_text = post_corpus2

In [111]:
# Assigning the list of target  
target_labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [112]:
# Calculating no of observation that are toxic, severe_toxic, obscene, threat, insult, indentity_hate
x = train.iloc[:, 2:].sum()

In [113]:
# Selecting independent variables
x = train.comment_text
test_x = test.comment_text

In [114]:
# Creating document term matrix
vect = TfidfVectorizer(max_features=20000, min_df =2)
x_dtm = vect.fit_transform(x)
text_x_dtm = vect.transform(test_x)

In [115]:
# Creating logistic regression model 
logreg = LogisticRegression(C =6.0)

In [117]:
# Predicting accuracy and probabilities 
for label in target_labels:
    # Here we will store each target column in y
    y = train[label]
    print('...Processing {}'.format(label))
    # Here we will train our model
    logreg.fit(x_dtm, y)
    # Here we will predict and classify the values of train datatset
    y_pred = logreg.predict(x_dtm)
    # Here we willcalculate the performance of our model
    performance(y,y_pred,label)
    test_y_prob = logreg.predict_proba(text_x_dtm)[:,1]
    # Here we just replace the sample dataset with the predicted ones
    #sample[label] = test_y_prob

...Processing toxic
Performance evaluation of toxic
Training accuracy is 0.972042269519102
Training precision is 0.9349455381233137
Training recall score is 0.7622616913801532
Training F1 score is 0.9122516260818723
...Processing severe_toxic
Performance evaluation of severe_toxic
Training accuracy is 0.9925973507132394
Training precision is 0.7317073170731707
Training recall score is 0.4195804195804196
Training F1 score is 0.7648012086325633
...Processing obscene
Performance evaluation of obscene
Training accuracy is 0.9861895548226889
Training precision is 0.9375977750738745
Training recall score is 0.793468667255075
Training F1 score is 0.926135438993128
...Processing threat
Performance evaluation of threat
Training accuracy is 0.9979789592423447
Training precision is 0.8604651162790697
Training recall score is 0.387434554973822
Training F1 score is 0.7666416555143851
...Processing insult
Performance evaluation of insult
Training accuracy is 0.9796015886320374
Training precision is 

In [ ]:
def performance(y, y_pred, label):
    print('Performance evaluation of {}'.format(label))
    print('Training accuracy is {}'.format(accuracy_score(y,y_pred)))
    print('Training precision is {}'.format(precision_score(y,y_pred)))
    print('Training recall score is {}'.format(recall_score(y,y_pred)))
    print('Training F1 score is {}'.format(f1_score(y,y_pred, average='macro')))
    

In [98]:
sample.head(25)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.999267,0.126952,0.996623,0.022925,0.797726,0.143898
1,0000247867823ef7,0.001910,0.000836,0.000854,0.000430,0.003141,0.001557
2,00013b17ad220c46,0.009638,0.000335,0.002768,0.000136,0.007034,0.001572
3,00017563c3f7919a,0.000930,0.001876,0.001102,0.000378,0.001022,0.000133
4,00017695ad8997eb,0.007715,0.003323,0.002067,0.000914,0.003181,0.001095
5,0001ea8717f6de06,0.003995,0.000362,0.002116,0.000079,0.007130,0.000527
6,00024115d4cbde0f,0.000607,0.000150,0.001219,0.000065,0.001279,0.000309
7,000247e83dcc1211,0.679917,0.001104,0.049360,0.003466,0.121804,0.001041
8,00025358d4737918,0.008196,0.001392,0.003297,0.001212,0.005000,0.002254
9,00026d1092fe71cc,0.002141,0.000439,0.003600,0.000493,0.008481,0.000927
